Conectando ao Banco de Dados Postgree

In [0]:
import requests
from pyspark.sql import Row
from pyspark.sql.functions import upper, trim,translate,col,input_file_name,split

In [0]:
driver = "org.postgresql.Driver"

database_host = "xxxxxx"
database_port = "5432" 
database_name = "xxxxx"
table = "pesquisa_satisfacao"
user = "xxxx"
password = "xxxxxx"

url = f"jdbc:postgresql://{database_host}:{database_port}/{database_name}"

df_pesquisa = (spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

In [0]:


listaCeps = [row["cep"] for row in df_pesquisa.select("cep").distinct().collect()]

def buscar_localizacao(cep):
    url = f"https://viacep.com.br/ws/{cep}/json/"
    localizacao = requests.get(url).json()

    return Row(
                cep = cep,
                localidade  =   localizacao.get("localidade"),
                uf  =   localizacao.get("uf"),
                estado  =   localizacao.get("estado"),
                regiao  =   localizacao.get("regiao"),
                ibge    =   localizacao.get("ibge")
            )
    

listaInfoCeps = [buscar_localizacao(cep) for cep in listaCeps]

df_ceps = spark.createDataFrame(listaInfoCeps)
df_ceps.display()
 



cep,localidade,uf,estado,regiao,ibge
59010200,null,null,null,null,null
30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200
38280000,Iturama,MG,Minas Gerais,Sudeste,3134400
64000120,Teresina,PI,Piauí,Nordeste,2211001
40010000,Salvador,BA,Bahia,Nordeste,2927408
69005070,Manaus,AM,Amazonas,Norte,1302603
66010090,null,null,null,null,null
88010400,Florianópolis,SC,Santa Catarina,Sul,4205407
01001000,São Paulo,SP,São Paulo,Sudeste,3550308
80010000,Curitiba,PR,Paraná,Sul,4106902


In [0]:
df_pesquisa_enriquecido = df_pesquisa.join(df_ceps, 
                                           df_pesquisa["cep"] == df_ceps["cep"],
                                           how='left').select(
                                               df_pesquisa['nome'],
                                               df_pesquisa['nota'],
                                               df_pesquisa['data_resposta'],
                                               df_ceps['*']
                                           )
df_pesquisa_enriquecido.display()

nome,nota,data_resposta,cep,localidade,uf,estado,regiao,ibge
Elisa Andrade,5,2025-03-10,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200
Luana Garcia,8,2025-02-01,66010090,null,null,null,null,null
Benicio Ribeiro,2,2025-03-28,66010090,null,null,null,null,null
Mariah Rios,9,2025-03-08,64000120,Teresina,PI,Piauí,Nordeste,2211001
Amanda Teixeira,5,2025-03-22,70040010,Brasília,DF,Distrito Federal,Centro-Oeste,5300108
Dra. Ester Mendes,3,2025-02-28,64000120,Teresina,PI,Piauí,Nordeste,2211001
Luiz Felipe Rios,8,2025-02-17,40010000,Salvador,BA,Bahia,Nordeste,2927408
Benicio Carvalho,4,2025-01-14,60060000,Fortaleza,CE,Ceará,Nordeste,2304400
Nicolas Camargo,2,2025-02-03,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200
Stephany Viana,2,2025-02-10,38280000,Iturama,MG,Minas Gerais,Sudeste,3134400


In [0]:
df_pesquisa_enriquecido.display()

nome,nota,data_resposta,cep,localidade,uf,estado,regiao,ibge
Elisa Andrade,5,2025-03-10,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200
Luana Garcia,8,2025-02-01,66010090,null,null,null,null,null
Benicio Ribeiro,2,2025-03-28,66010090,null,null,null,null,null
Mariah Rios,9,2025-03-08,64000120,Teresina,PI,Piauí,Nordeste,2211001
Amanda Teixeira,5,2025-03-22,70040010,Brasília,DF,Distrito Federal,Centro-Oeste,5300108
Dra. Ester Mendes,3,2025-02-28,64000120,Teresina,PI,Piauí,Nordeste,2211001
Luiz Felipe Rios,8,2025-02-17,40010000,Salvador,BA,Bahia,Nordeste,2927408
Benicio Carvalho,4,2025-01-14,60060000,Fortaleza,CE,Ceará,Nordeste,2304400
Nicolas Camargo,2,2025-02-03,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200
Stephany Viana,2,2025-02-10,38280000,Iturama,MG,Minas Gerais,Sudeste,3134400


In [0]:
df_classificacao_nomes = spark.read.format("delta").load("/dbfs/tmp/classificacao_nomes_delta")

In [0]:
df_classificacao_nomes.display()

genero,feminino,masculino,quantidade,nome,percentual_indice
F,11702212,39443,11741655,MARIA,1.00
M,22760,5743253,5766013,JOSE,1.00
F,3148328,10452,3158780,ANA,1.00
M,12184,2972031,2984215,JOAO,1.00
M,8854,2568963,2577817,ANTONIO,1.00
M,40320,2040921,2081241,LUIZ,0.98
M,7000,1765516,1772516,FRANCISCO,1.00
M,6070,1485283,1491353,CARLOS,1.00
M,5400,1423435,1428835,PAULO,1.00
M,6048,1213590,1219638,PEDRO,1.00


In [0]:


df_pesquisa_enriquecido = df_pesquisa_enriquecido.withColumn("primeiro_nome", split(df_pesquisa_enriquecido.nome, " ").getItem(0))

df_pesquisa_enriquecido.display()

nome,nota,data_resposta,cep,localidade,uf,estado,regiao,ibge,primeiro_nome
Elisa Andrade,5,2025-03-10,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200,Elisa
Luana Garcia,8,2025-02-01,66010090,null,null,null,null,null,Luana
Benicio Ribeiro,2,2025-03-28,66010090,null,null,null,null,null,Benicio
Mariah Rios,9,2025-03-08,64000120,Teresina,PI,Piauí,Nordeste,2211001,Mariah
Amanda Teixeira,5,2025-03-22,70040010,Brasília,DF,Distrito Federal,Centro-Oeste,5300108,Amanda
Dra. Ester Mendes,3,2025-02-28,64000120,Teresina,PI,Piauí,Nordeste,2211001,Dra.
Luiz Felipe Rios,8,2025-02-17,40010000,Salvador,BA,Bahia,Nordeste,2927408,Luiz
Benicio Carvalho,4,2025-01-14,60060000,Fortaleza,CE,Ceará,Nordeste,2304400,Benicio
Nicolas Camargo,2,2025-02-03,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200,Nicolas
Stephany Viana,2,2025-02-10,38280000,Iturama,MG,Minas Gerais,Sudeste,3134400,Stephany


In [0]:


df_pesquisa_enriquecido = df_pesquisa_enriquecido.withColumn(
    "primeiro_nome",
    translate("primeiro_nome", "áéíóúàèìòùâêîôûãõäëïöüçÁÉÍÓÚÀÈÌÒÙÂÊÎÔÛÃÕÄËÏÖÜÇ", 
                    "aeiouaeiouaeiouaoaeioucAEIOUAEIOUAEIOUAOAEIOUC")
)

df_pesquisa_enriquecido = df_pesquisa_enriquecido.withColumn("primeiro_nome",trim(upper(df_pesquisa_enriquecido.primeiro_nome)))

In [0]:
df_pesquisa_enriquecido.display()

nome,nota,data_resposta,cep,localidade,uf,estado,regiao,ibge,primeiro_nome
Elisa Andrade,5,2025-03-10,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200,ELISA
Luana Garcia,8,2025-02-01,66010090,null,null,null,null,null,LUANA
Benicio Ribeiro,2,2025-03-28,66010090,null,null,null,null,null,BENICIO
Mariah Rios,9,2025-03-08,64000120,Teresina,PI,Piauí,Nordeste,2211001,MARIAH
Amanda Teixeira,5,2025-03-22,70040010,Brasília,DF,Distrito Federal,Centro-Oeste,5300108,AMANDA
Dra. Ester Mendes,3,2025-02-28,64000120,Teresina,PI,Piauí,Nordeste,2211001,DRA.
Luiz Felipe Rios,8,2025-02-17,40010000,Salvador,BA,Bahia,Nordeste,2927408,LUIZ
Benicio Carvalho,4,2025-01-14,60060000,Fortaleza,CE,Ceará,Nordeste,2304400,BENICIO
Nicolas Camargo,2,2025-02-03,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200,NICOLAS
Stephany Viana,2,2025-02-10,38280000,Iturama,MG,Minas Gerais,Sudeste,3134400,STEPHANY


In [0]:
df_indicadores_pesquisa = df_pesquisa_enriquecido.join(df_classificacao_nomes,
                                                       df_pesquisa_enriquecido["primeiro_nome"] == df_classificacao_nomes["nome"], 
                                                       how = "left").select(
                                                            df_pesquisa_enriquecido["*"], 
                                                            df_classificacao_nomes["genero"],
                                                            df_classificacao_nomes["percentual_indice"]  
                                                        )
df_indicadores_pesquisa.display()

nome,nota,data_resposta,cep,localidade,uf,estado,regiao,ibge,primeiro_nome,genero,percentual_indice
Elisa Andrade,5,2025-03-10,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200,ELISA,null,null
Luana Garcia,8,2025-02-01,66010090,null,null,null,null,null,LUANA,F,1.00
Benicio Ribeiro,2,2025-03-28,66010090,null,null,null,null,null,BENICIO,M,1.00
Mariah Rios,9,2025-03-08,64000120,Teresina,PI,Piauí,Nordeste,2211001,MARIAH,null,null
Amanda Teixeira,5,2025-03-22,70040010,Brasília,DF,Distrito Federal,Centro-Oeste,5300108,AMANDA,F,1.00
Dra. Ester Mendes,3,2025-02-28,64000120,Teresina,PI,Piauí,Nordeste,2211001,DRA.,null,null
Luiz Felipe Rios,8,2025-02-17,40010000,Salvador,BA,Bahia,Nordeste,2927408,LUIZ,M,0.98
Benicio Carvalho,4,2025-01-14,60060000,Fortaleza,CE,Ceará,Nordeste,2304400,BENICIO,M,1.00
Nicolas Camargo,2,2025-02-03,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200,NICOLAS,M,0.99
Stephany Viana,2,2025-02-10,38280000,Iturama,MG,Minas Gerais,Sudeste,3134400,STEPHANY,null,null


In [0]:
df_indicadores_pesquisa.write.format("delta").mode("overwrite").save("/dbfs/tmp/pesquisa_satisfacao")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW vw_indicadores_pesquisa AS
SELECT * 
FROM delta.`/dbfs/tmp/pesquisa_satisfacao`

In [0]:
%sql
select * from vw_indicadores_pesquisa

nome,nota,data_resposta,cep,localidade,uf,estado,regiao,ibge,primeiro_nome,genero,percentual_indice
Elisa Andrade,5,2025-03-10,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200,ELISA,null,null
Luana Garcia,8,2025-02-01,66010090,null,null,null,null,null,LUANA,F,1.00
Benicio Ribeiro,2,2025-03-28,66010090,null,null,null,null,null,BENICIO,M,1.00
Mariah Rios,9,2025-03-08,64000120,Teresina,PI,Piauí,Nordeste,2211001,MARIAH,null,null
Amanda Teixeira,5,2025-03-22,70040010,Brasília,DF,Distrito Federal,Centro-Oeste,5300108,AMANDA,F,1.00
Dra. Ester Mendes,3,2025-02-28,64000120,Teresina,PI,Piauí,Nordeste,2211001,DRA.,null,null
Luiz Felipe Rios,8,2025-02-17,40010000,Salvador,BA,Bahia,Nordeste,2927408,LUIZ,M,0.98
Benicio Carvalho,4,2025-01-14,60060000,Fortaleza,CE,Ceará,Nordeste,2304400,BENICIO,M,1.00
Nicolas Camargo,2,2025-02-03,30130010,Belo Horizonte,MG,Minas Gerais,Sudeste,3106200,NICOLAS,M,0.99
Stephany Viana,2,2025-02-10,38280000,Iturama,MG,Minas Gerais,Sudeste,3134400,STEPHANY,null,null


Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

In [0]:
%sql
SELECT 
  COUNT(1) AS quantidade_respostas,
  date_format(data_resposta, 'MM/yyyy') AS mes_ano
FROM vw_indicadores_pesquisa
GROUP BY date_format(data_resposta, 'MM/yyyy')
ORDER BY mes_ano

quantidade_respostas,mes_ano
30,01/2025
32,02/2025
33,03/2025
5,04/2025


In [0]:
%sql
SELECT 
  regiao,
  ROUND(AVG(nota), 2) AS media_notas
FROM vw_indicadores_pesquisa
GROUP BY regiao
ORDER BY media_notas DESC

regiao,media_notas
Centro-Oeste,7.25
Norte,6.38
Sul,6.2
Nordeste,5.65
Sudeste,5.15
null,5.05
